<a href="https://colab.research.google.com/github/TimL2/Group_Project_2_ML_Bot/blob/main/ML_Trading_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install alpaca-trade-api

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 3.1 MB/s 
     |████████████████████████████████| 112 kB 27.0 MB/s 
     |████████████████████████████████| 1.1 MB 50.8 MB/s 
     |████████████████████████████████| 299 kB 45.3 MB/s 
  Attempting uninstall: msgpack
    Found existing installation: msgpack 1.0.4
    Uninstalling msgpack-1.0.4:
      Successfully uninstalled msgpack-1.0.4
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.8.3
    Uninstalling aiohttp-3.8.3:
      Successfully uninstalled aiohttp-3.8.3


In [7]:
# Initial imports
import pandas as pd
import alpaca_trade_api as tradeapi

In [11]:
ALPACA_API_KEY = 'PKETI0Q5C8PPUXVNHJFJ'
ALPACA_SECRET_KEY = 'WQxmEpBCoMXydCCg0G8cUe6hGLpgaAfrvedZ09Fy'
ALPACA_API_BASE_URL = "https://paper-api.alpaca.markets"

In [12]:
# Create a connection to the API 
api = tradeapi.REST(ALPACA_API_KEY, ALPACA_SECRET_KEY, ALPACA_API_BASE_URL, api_version="v2")

In [13]:
# Set signal variable
signal = 1

In [14]:
 # Create buy signal, num shares and ticker
if signal == 1:
    orderSide = "buy"
else:
    orderSide = "sell"

In [15]:
# Set the ticket symbol and the number of shares to buy
ticker = "TSLA"
number_of_shares = 1

In [16]:
 # Make API call
prices = api.get_bars(ticker, "1Min").df

# Reorganize the DataFrame
prices = pd.concat([prices], axis=1, keys=["TSLA"])

# Get final closing price
limit_amount = prices["TSLA"]["close"][-1]

In [17]:
prices

TSLA                                             \
                             open    high     low   close volume trade_count   
timestamp                                                                      
2022-10-17 08:00:00+00:00  210.32  210.32  208.05  208.71  21779         543   
2022-10-17 08:01:00+00:00  208.77  209.14  208.52  208.82  10910         409   
2022-10-17 08:02:00+00:00  208.90  208.90  208.38  208.50  10515         422   
2022-10-17 08:03:00+00:00  208.49  208.50  208.19  208.32   5429         248   
2022-10-17 08:04:00+00:00  208.21  208.39  208.20  208.39   3965         223   
...                           ...     ...     ...     ...    ...         ...   
2022-10-17 23:55:00+00:00  222.80  222.87  222.71  222.71   7792         123   
2022-10-17 23:56:00+00:00  222.72  222.75  222.70  222.75   3651          91   
2022-10-17 23:57:00+00:00  222.75  222.80  222.75  222.75   5887          78   
2022-10-17 23:58:00+00:00  222.80  222.80  222.72  222.75   8468         114   
2022-10-17 23:59:00+00:00  222.78  222.80  222.72  222.75  12511         191   

                                       
                                 vwap  
timestamp                              
2022-10-17 08:00:00+00:00  209.101876  
2022-10-17 08:01:00+00:00  208.879309  
2022-10-17 08:02:00+00:00  208.624815  
2022-10-17 08:03:00+00:00  208.388886  
2022-10-17 08:04:00+00:00  208.292076  
...                               ...  
2022-10-17 23:55:00+00:00  222.799405  
2022-10-17 23:56:00+00:00  222.733232  
2022-10-17 23:57:00+00:00  222.773214  
2022-10-17 23:58:00+00:00  222.764763  
2022-10-17 23:59:00+00:00  222.752236  

[918 rows x 7 columns]

In [18]:
limit_amount

222.75